In [2]:
import pandas as pd
import librosa
import os
import numpy as np
metadata = pd.read_csv('/kaggle/input/urbansound8k/UrbanSound8K.csv')

metadata.head()

,slice_file_name,fsID,start,end,salience,fold,classID,class
0,100032-3-0-0.wav,100032,0.0,0.317551,1,5,3,dog_bark
1,100263-2-0-117.wav,100263,58.5,62.500000,1,5,2,children_playing
2,100263-2-0-121.wav,100263,60.5,64.500000,1,5,2,children_playing
3,100263-2-0-126.wav,100263,63.0,67.000000,1,5,2,children_playing
4,100263-2-0-137.wav,100263,68.5,72.500000,1,5,2,children_playing


In [3]:
print(metadata["class"].value_counts())
class_names = [ 'dog_bark', 
        'children_playing', 
        'air_conditioner', 
        'street_music', 
        'jackhammer', 
        'engine_idling', 
        'drilling', 
        'siren', 
        'car_horn', 
        'gun_shot']

class
dog_bark            1000
children_playing    1000
air_conditioner     1000
street_music        1000
jackhammer          1000
engine_idling       1000
drilling            1000
siren                929
car_horn             429
gun_shot             374
Name: count, dtype: int64


In [5]:
import os
import numpy as np
import pandas as pd
import librosa
from concurrent.futures import ThreadPoolExecutor

def process_audio_file(file_path, sample_rate):
    audio, sr = librosa.load(file_path, sr=sample_rate)
    mfccs = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=13)
    mfccs = np.mean(mfccs.T, axis=0)
    return mfccs

def load_audio_files(dataframe, sample_rate=22050, duration=5):
    audio_data = []
    labels = []

    with ThreadPoolExecutor() as executor:
        futures = []
        for index, row in dataframe.iterrows():
            file_path = os.path.join('/kaggle/input/urbansound8k', 'fold' + str(row['fold']), row['slice_file_name'])
            futures.append(executor.submit(process_audio_file, file_path, sample_rate))
            labels.append(row['classID'])

        for future in futures:
            audio_data.append(future.result())

    return np.array(audio_data), np.array(labels)


X, y = load_audio_files(metadata)
print(X.shape, y.shape)


/opt/conda/lib/python3.10/site-packages/librosa/core/spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=1323
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/librosa/core/spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=1103
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/librosa/core/spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=1523
  warnings.warn(


(8732, 13) (8732,)


In [6]:
print(y[1])#labels
print(X[1])#Audio Data

2
[-417.00525     99.336624   -42.995586    51.073326     9.853778
    7.9696927   11.197088     1.9291176    7.0303974    4.270228
   -3.5847254   13.890946    -2.7644932]


In [7]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))


In [8]:
import tensorflow as tf
from tensorflow.keras import layers, models

input_shape = (X_train.shape[1], X_train.shape[2])  
model = models.Sequential([
    layers.LSTM(64, input_shape=input_shape, return_sequences=True),
    layers.LSTM(32),
    layers.Dense(64, activation='relu'),
    layers.Dense(128, activation='relu'),
    layers.Dense(10, activation='softmax')
    
])


model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


/opt/conda/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [9]:
history = model.fit(X_train, y_train, epochs=25, batch_size=32, validation_split=0.2)


Epoch 1/25
175/175 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - accuracy: 0.2330 - loss: 2.1338 - val_accuracy: 0.4202 - val_loss: 1.6396
Epoch 2/25
175/175 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.4812 - loss: 1.5337 - val_accuracy: 0.5132 - val_loss: 1.4404
Epoch 3/25
175/175 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.5516 - loss: 1.3463 - val_accuracy: 0.5634 - val_loss: 1.3187
Epoch 4/25
175/175 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.6001 - loss: 1.2184 - val_accuracy: 0.6013 - val_loss: 1.2272
Epoch 5/25
175/175 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.6407 - loss: 1.0883 - val_accuracy: 0.5891 - val_loss: 1.2302
Epoch 6/25
175/175 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.6736 - loss: 1.0228 - val_accuracy: 0.6328 - val_loss: 1.1384
Epoch 7/25
175/175 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.6957 - loss: 0.9505 - val_accuracy: 0.6399 - val_loss: 1.0999
Epoch 8/25
175/175 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.7039 - loss: 0.9276 - val_accuracy: 0.

In [10]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {accuracy:}')


55/55 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7376 - loss: 0.9572
Test Accuracy: 0.733256995677948


VAL_LOSS_RNN = 0.73